## **Project Overview**

The restaurant aggregator industry is becoming increasingly reliant on sophisticated technology solutions to stay competitive in the fast-paced food delivery market. In this context, providing users with highly personalized and contextually relevant food recommendations is crucial for enhancing user experience and fostering brand loyalty. Traditional recommendation systems, predominantly based on textual data, often fail to capture the comprehensive and nuanced preferences of users, especially when it comes to visual appeal and presentation of dishes. A multimodal approach that incorporates both text and images can vastly improve the accuracy and personalization of recommendations.

This project aims to design and implement a Multimodal Retrieval-Augmented Generation (RAG) system for a restaurant aggregator app, enhancing its capability to deliver precise food recommendations tailored to individual preferences and dietary needs. The system will integrate and process multiple data types—textual descriptions and visual content—from restaurants to generate personalized suggestions. Utilizing technologies such as Amazon S3 for data storage, Amazon Bedrock for image summarization, and FAISS for efficient similarity search, the system will encode and retrieve vectorized data representations. A user interface powered by Streamlit will facilitate interactive and user-friendly querying, ultimately leading to dynamic and context-aware recommendation generation. 

## **Approach**

* Data Reading and Preprocessing:
    * Data Storage: Utilize Amazon S3 for scalable and reliable object storage of the collected data and read the images and metadata.

* Data Processing:
    * Image Data: Use Anthropic Claude-Sonnet model to generate image descriptions 

* Vector Database:
    * Storage and Retrieval: Use Amazon Titan Embeddings and FAISS for storing and efficiently retrieving vectorized data, enabling fast and scalable search capabilities for the recommendation engine.

* Recommendation Engine: 
    * Utilizing Anthropic Claude Sonnet Multimodal Model: To create conversational chatbot and generate recommendations from Vector DB results.

* Streamlit API Development and Integration:
    * User Interface: Develop an interactive user interface using Streamlit, which will serve the frontend for users to interact with the system.
    * Functionalities: Users will be able to interact with the chatbot to search text or image inputs.


## **Solution Architecture**

![image](reference-images/notebook/architecture.png)

## **Multimodal Data**

Multimodal data refers to information collected in various forms, including text, images, audio, video, and even sensor data, which are integrated and analyzed together to enhance the performance of artificial intelligence (AI) systems. This integration allows AI models to process and understand complex scenarios more like humans do, by interpreting different types of data concurrently. The significance of multimodal data in real-world applications is profound, enabling more nuanced and context-aware AI solutions. For example, in healthcare, multimodal data can combine medical images, patient records, and doctor's audio notes to improve diagnostic accuracy. In customer service, text from chat interactions, voice data from calls, and video data can be analyzed together to enhance response quality and service personalization. The ability to merge and interpret this varied data leads to richer insights and more effective AI applications across diverse fields.

![image](reference-images/notebook/modalities.png)

## Importing Libraries

In [7]:
import pandas as pd
import boto3
from utils import *
import base64
import os
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

## Data Fetch

In [8]:
# Initialize a session using Amazon S3
s3_client = boto3.client('s3', region_name='your-region', 
                         aws_access_key_id='your-access-key-id', 
                         aws_secret_access_key='your-secret-access-key')

In [9]:
s3_client = boto3.client('s3', region_name='ap-south-1')

In [10]:
def fetch_csv_from_s3(bucket_name, file_key):
    """
    Fetches a CSV file from S3 and converts it into a Pandas DataFrame.
    
    :param bucket_name: Name of the S3 bucket
    :param file_key: Key (path) to the CSV file in the bucket
    :return: DataFrame containing the CSV data
    """
    # Fetch the CSV file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    
    # Read the CSV file content
    csv_content = response['Body'].read().decode('utf-8')
    
    # Use StringIO to convert the CSV string into a file-like object
    csv_buffer = StringIO(csv_content)
    
    # Load the CSV data into a DataFrame
    df = pd.read_csv(csv_buffer)
    
    return df

In [11]:
# usage
bucket_name = 'multimodal-food-recommendation'
file_key = 'restaurants_menu_data.csv'

df = fetch_csv_from_s3(bucket_name, file_key)

# use this code if reading data from local folder
# df = pd.read_csv("data/restaurants_menu_data.csv")
df.head()

,restaurant_id,restaurant_name,cuisine,menu_item_id,menu_item_name,ingredients,protein,carbs,fats,calories,dietary_warnings,vegetarian_or_nonveg,image_path,average_rating,price,serves
0,R001,La Bella Italia,Italian,R001M001,Margherita Pizza,"tomatoes, mozzarella cheese, basil, olive oil,...",12,30,15,350,NaN,Vegetarian,images/R001/R001M001.png,4.5,12,1-2
1,R001,La Bella Italia,Italian,R001M002,Spaghetti Carbonara,"spaghetti, eggs, cheese, pancetta, black pepper",18,40,20,400,"Contains eggs, Contains dairy",Non-Vegetarian,images/R001/R001M002.png,4.0,10,1-2
2,R001,La Bella Italia,Italian,R001M003,Lasagna,"pasta sheets, ground beef, ricotta cheese, moz...",25,35,22,450,Contains dairy,Non-Vegetarian,images/R001/R001M003.png,4.6,16,1-2
3,R001,La Bella Italia,Italian,R001M004,Bruschetta,"bread, tomatoes, garlic, basil, olive oil",4,15,5,120,NaN,Vegetarian,images/R001/R001M004.png,3.8,8,1
4,R001,La Bella Italia,Italian,R001M005,Tiramisu,"ladyfingers, coffee, mascarpone cheese, cocoa ...",6,25,15,300,"Contains dairy, Contains eggs",Vegetarian,images/R001/R001M005.png,3.1,12,1


## Creating Bedrock Runtime

In [12]:
import boto3
bedrock = boto3.client('bedrock-runtime')

In [13]:
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings

model_kwargs =  {
    "max_tokens": 2048,
    "temperature": 0.0,
    "stop_sequences": ["\n\nHuman"],
}

llm = BedrockChat(
    client=bedrock,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=model_kwargs,
)

embeddings=BedrockEmbeddings(
    client=bedrock,
    model_id="amazon.titan-embed-text-v2:0"
)

## Base64 Encoding

In [15]:
def encode_image_from_s3(bucket_name, image_path):
    """
    Fetches an image from S3 and encodes it in base64.

    :param bucket_name: The name of the S3 bucket.
    :param image_path: The relative path to the image in the S3 bucket.
    :return: The base64-encoded string of the image.
    """
    # Fetch the image from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=image_path)
    
    # Read the image content as binary
    image_content = response['Body'].read()

    # Encode the image content to a base64 string
    encoded_image = base64.b64encode(image_content).decode('utf-8')
    
    return encoded_image

In [16]:
# Use this function if you are using images from local folder instead of S3 bucket
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')
    
# df['encoded_image'] = df['image_path'].apply(encode_image)

In [17]:
# Create a new column 'encoded_image' by applying the encode_image_from_s3 function
df['encoded_image'] = df['image_path'].apply(lambda x: encode_image_from_s3(bucket_name, x))

In [18]:
df.head(1)

,restaurant_id,restaurant_name,cuisine,menu_item_id,menu_item_name,ingredients,protein,carbs,fats,calories,dietary_warnings,vegetarian_or_nonveg,image_path,average_rating,price,serves,encoded_image
0,R001,La Bella Italia,Italian,R001M001,Margherita Pizza,"tomatoes, mozzarella cheese, basil, olive oil,...",12,30,15,350,NaN,Vegetarian,images/R001/R001M001.png,4.5,12,1-2,iVBORw0KGgoAAAANSUhEUgAABD0AAALJCAYAAAC3J1hNAA...
